# Exercice 1

In [1]:
gamma_mult <- function(x, p) {
    π <- pi^(p * (p - 1.0) / 4.0)
    Γ <- gamma(x - seq_len(p) / 2.0 + 0.5)

    π * prod(Γ)
}

In [2]:
gamma_mult(6, 5)

[1] 3219542192

# Exercice 2

In [3]:
det_update_chol <- function(x, d, R) {
    y <- backsolve(R, x, transpose = TRUE)

    as.numeric((1 + crossprod(y)) * d)
}

In [4]:
M <- crossprod(matrix(rnorm(2.0^2L), 2L))
d <- det(M)
R <- chol(M)
x <- 1:2

det_update_chol(x, d, R)
det(M + tcrossprod(x))

[1] 5.410401

[1] 5.410401

# Exercice 3

In [5]:
wishart_pert <- function(n, M, m, Σ) {
    p <- nrow(M)
    dM <- det(M)
    dΣ <- det(Σ)
    Σ_inv <- solve(Σ)
    Γ = gamma_mult(0.5 * m, p)
    R <- chol(M)
    
    denum <- 2.0^(0.5 * m * p) * dΣ^(0.5 * m) * Γ

    replicate(n, simplify = FALSE, {
        x <- rnorm(p)
        newM <- M + tcrossprod(x)

        num <- det_update_chol(x, dM, R)^((m - p - 1.0) / 2.0) *
            exp(-0.5 * sum(diag(Σ_inv %*% newM)))

        fM <- num / denum

        list(M = newM, fM = fM)
    })
}

In [6]:
set.seed(666)
wishart_pert(2L, M, 4.0, diag(2L))

0.8048792,0.8649487
0.8649487,5.8974363
0.3635222,-1.372759
-1.3727592,5.953276


## Commentaire
Il aurait été préférable de travailler sur l'échelle logarithmique pour la densité. On a
$$
\log f(\boldsymbol M) = (m - p - 1) \log |\boldsymbol M| - \text{tr}(\boldsymbol{\Sigma}^{-1} \boldsymbol M) -
    m(p \log 2 + log |\boldsymbol{\Sigma}|) + 2 \log \Gamma_p \left( \frac m 2 \right)
$$
où
$$
    \log \Gamma_p(x) = \frac{p (p - 1)}{4} \log \pi + \sum_{k = 1}^p \log \Gamma \left( m - \frac{k - 1}{2} \right)
$$
qu'on pourrait spécialiser d'avantage au besoin. Implémenté ci-dessous.

In [7]:
gamma_mult <- function(x, p, log = FALSE) {
    log_π <- (p * (p - 1.0) / 4.0) * log(pi)
    log_Γ <- log(gamma(x - seq_len(p) / 2.0 + 0.5))

    res  <- log_π + sum(log_Γ)
    ifelse(log, res, exp(res))
}

wishart_pert <- function(n, M, m, Σ, log = FALSE) {
    p <- nrow(M)
    dM <- det(M)
    log_dΣ <- log(det(Σ))
    Σ_inv <- solve(Σ)
    log_Γ = gamma_mult(0.5 * m, p, log = TRUE)
    R <- chol(M)
    
    log_2 <- log(2)
    log_denum <- m * (p * log_2 + log_dΣ) + 2 * log_Γ

    replicate(n, simplify = FALSE, {
        x <- rnorm(p)
        newM <- M + tcrossprod(x)

        log_num <- (m - p - 1.0) * log(det_update_chol(x, dM, R)) -
            sum(diag(Σ_inv %*% newM))
        
        log_fM <- log_num - log_denum

        list(M = newM, fM = ifelse(log, log_fM, exp(log_fM)))
    })
}

In [8]:
gamma_mult(6, 5)

[1] 3219542192

In [9]:
set.seed(666)
wishart_pert(2L, M, 4.0, diag(2L))

0.8048792,0.8649487
0.8649487,5.8974363
0.3635222,-1.372759
-1.3727592,5.953276
